#3.3 Code Tutorial: Configurations for Deep Neural Networks

This code tutorial provides hands-on experience on how to configure your first deep neural network. You will start by exploring the fashion MNIST dataset. Next, you will look at how to avoid overfitting by going through a few popular regularization techniques for large neural networks. To do this, you will try to integrate L1 and L2 regularization into your network and see how well they work. Additionally, you will try a few variants of dropout principles including Dropout, Alpha Dropout, and MC Dropout layers.

# Setup

First, let's import a few common modules, ensure MatplotLib plots figures inline and prepare a function to save the figures. We also check that Python 3.5 or later is installed (although Python 2.x may work, it is deprecated so we strongly recommend you use Python 3 instead), as well as Scikit-Learn ≥0.20 and TensorFlow ≥2.0-preview.

In [1]:
# Python ≥3.5 is required
import sys
#assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
#assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0-preview is required
import tensorflow as tf
from tensorflow import keras
%tensorflow_version 2.x
print(tf.__version__)
#assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
2.15.0


In [2]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

4422102/4422102 [==============================] - 0s 0us/step


In [3]:
pixel_means = X_train.mean(axis=0, keepdims=True)
pixel_stds = X_train.std(axis=0, keepdims=True)
X_train_scaled = (X_train - pixel_means) / pixel_stds
X_valid_scaled = (X_valid - pixel_means) / pixel_stds
X_test_scaled = (X_test - pixel_means) / pixel_stds

In [5]:
print(X_train.shape)

(55000, 28, 28)


## Unregularized Network

In [4]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="elu",
                       kernel_initializer="he_normal"),
    keras.layers.Dense(100, activation="elu",
                       kernel_initializer="he_normal"),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
1719/1719 [==============================] - 16s 8ms/step - loss: 0.4509 - accuracy: 0.8424 - val_loss: 0.3671 - val_accuracy: 0.8682
Epoch 2/2
1719/1719 [==============================] - 13s 7ms/step - loss: 0.3271 - accuracy: 0.8788 - val_loss: 0.3386 - val_accuracy: 0.8738


# Avoiding Overfitting Through Regularization

## $\ell_1$ and $\ell_2$ regularization

In [6]:
layer = keras.layers.Dense(100, activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=keras.regularizers.l2(0.01))
# or l1(0.1) for ℓ1 regularization with a factor or 0.1
# or l1_l2(0.1, 0.01) for both ℓ1 and ℓ2 regularization, with factors 0.1 and 0.01 respectively

In [7]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="elu",
                       kernel_initializer="he_normal",
                       kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(100, activation="elu",
                       kernel_initializer="he_normal",
                       kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(10, activation="softmax",
                       kernel_regularizer=keras.regularizers.l2(0.01))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
1719/1719 [==============================] - 16s 8ms/step - loss: 1.5722 - accuracy: 0.8135 - val_loss: 0.7413 - val_accuracy: 0.8338
Epoch 2/2
1719/1719 [==============================] - 14s 8ms/step - loss: 0.7222 - accuracy: 0.8251 - val_loss: 0.6920 - val_accuracy: 0.8406


In [8]:
from functools import partial

RegularizedDense = partial(keras.layers.Dense,
                           activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=keras.regularizers.l2(0.01))

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(300),
    RegularizedDense(100),
    RegularizedDense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
1719/1719 [==============================] - 15s 8ms/step - loss: 1.5963 - accuracy: 0.8131 - val_loss: 0.7611 - val_accuracy: 0.8070
Epoch 2/2
1719/1719 [==============================] - 14s 8ms/step - loss: 0.7280 - accuracy: 0.8250 - val_loss: 0.7075 - val_accuracy: 0.8284


## Dropout

In [9]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2;
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
1719/1719 [==============================] - 16s 8ms/step - loss: 0.5784 - accuracy: 0.8020 - val_loss: 0.3595 - val_accuracy: 0.8684
Epoch 2/2
1719/1719 [==============================] - 16s 9ms/step - loss: 0.4222 - accuracy: 0.8450 - val_loss: 0.3580 - val_accuracy: 0.8712


## Alpha Dropout

In [10]:
tf.random.set_seed(42)
np.random.seed(42)

In [11]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.AlphaDropout(rate=0.1),
    keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.AlphaDropout(rate=0.1),
    keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.AlphaDropout(rate=0.1),
    keras.layers.Dense(10, activation="softmax")
])
optimizer = keras.optimizers.SGD(lr=0.01, momentum=0.9, nesterov=True)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
n_epochs = 2;
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
1719/1719 [==============================] - 13s 7ms/step - loss: 0.5708 - accuracy: 0.7935 - val_loss: 0.4386 - val_accuracy: 0.8622
Epoch 2/2
1719/1719 [==============================] - 12s 7ms/step - loss: 0.4689 - accuracy: 0.8291 - val_loss: 0.4207 - val_accuracy: 0.8664


In [12]:
model.evaluate(X_test_scaled, y_test)

313/313 [==============================] - 1s 4ms/step - loss: 0.4620 - accuracy: 0.8500


[0.46202757954597473, 0.8500000238418579]

In [13]:
model.evaluate(X_train_scaled, y_train)

1719/1719 [==============================] - 4s 2ms/step - loss: 0.3917 - accuracy: 0.8658


[0.3917275667190552, 0.8658363819122314]

In [14]:
history = model.fit(X_train_scaled, y_train)

1719/1719 [==============================] - 11s 7ms/step - loss: 0.4373 - accuracy: 0.8393


## MC Dropout

In [15]:
tf.random.set_seed(42)
np.random.seed(42)

In [16]:
y_probas = np.stack([model(X_test_scaled, training=True)
                     for sample in range(100)])
y_proba = y_probas.mean(axis=0)
y_std = y_probas.std(axis=0)

In [17]:
np.round(model.predict(X_test_scaled[:1]), 2)

1/1 [==============================] - 0s 143ms/step


array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.02, 0.  , 0.19, 0.  , 0.79]],
      dtype=float32)

In [18]:
np.round(y_probas[:, :1], 2)

array([[[0.  , 0.  , 0.  , 0.  , 0.  , 0.02, 0.  , 0.46, 0.  , 0.52]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.02, 0.  , 0.86, 0.02, 0.1 ]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.17, 0.  , 0.14, 0.  , 0.7 ]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.03, 0.  , 0.64, 0.  , 0.33]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.11, 0.  , 0.56, 0.  , 0.34]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.71, 0.  , 0.13, 0.  , 0.16]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.02, 0.  , 0.04, 0.  , 0.95]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.22, 0.  , 0.19, 0.  , 0.59]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.31, 0.  , 0.16, 0.  , 0.53]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.02, 0.  , 0.07, 0.  , 0.91]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.22, 0.  , 0.12, 0.  , 0.66]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.1 , 0.  , 0.18, 0.  , 0.72]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.03, 0.  , 0.05, 0.  , 0.92]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.41, 0.  , 0.05, 0.  , 0

In [19]:
np.round(y_proba[:1], 2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.13, 0.  , 0.33, 0.  , 0.54]],
      dtype=float32)

In [20]:
y_std = y_probas.std(axis=0)
np.round(y_std[:1], 2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.16, 0.  , 0.23, 0.  , 0.24]],
      dtype=float32)

In [21]:
y_pred = np.argmax(y_proba, axis=1)

In [22]:
accuracy = np.sum(y_pred == y_test) / len(y_test)
accuracy

0.86

In [23]:
class MCDropout(keras.layers.Dropout):
    def call(self, inputs):
        return super().call(inputs, training=True)

class MCAlphaDropout(keras.layers.AlphaDropout):
    def call(self, inputs):
        return super().call(inputs, training=True)

In [24]:
tf.random.set_seed(42)
np.random.seed(42)

In [25]:
mc_model = keras.models.Sequential([
    MCAlphaDropout(layer.rate) if isinstance(layer, keras.layers.AlphaDropout) else layer
    for layer in model.layers
])

In [26]:
mc_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_4 (Flatten)         (None, 784)               0         
                                                                 
 mc_alpha_dropout (MCAlphaD  (None, 784)               0         
 ropout)                                                         
                                                                 
 dense_13 (Dense)            (None, 300)               235500    
                                                                 
 mc_alpha_dropout_1 (MCAlph  (None, 300)               0         
 aDropout)                                                       
                                                                 
 dense_14 (Dense)            (None, 100)               30100     
                                                                 
 mc_alpha_dropout_2 (MCAlph  (None, 100)              

In [28]:
optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
mc_model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [29]:
mc_model.set_weights(model.get_weights())

Now we can use the model with MC Dropout:

In [30]:
np.round(np.mean([mc_model.predict(X_test_scaled[:1]) for sample in range(100)], axis=0), 2)

1/1 [==============================] - 0s 23ms/step


array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.12, 0.  , 0.3 , 0.  , 0.57]],
      dtype=float32)

## Max norm

In [31]:
layer = keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal",
                           kernel_constraint=keras.constraints.max_norm(1.))

In [32]:
MaxNormDense = partial(keras.layers.Dense,
                       activation="selu", kernel_initializer="lecun_normal",
                       kernel_constraint=keras.constraints.max_norm(1.))

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    MaxNormDense(300),
    MaxNormDense(100),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
1719/1719 [==============================] - 19s 10ms/step - loss: 0.4771 - accuracy: 0.8336 - val_loss: 0.3581 - val_accuracy: 0.8710
Epoch 2/2
1719/1719 [==============================] - 15s 9ms/step - loss: 0.3588 - accuracy: 0.8709 - val_loss: 0.3965 - val_accuracy: 0.8590
